# Search Algorithms
The following notebook contains class definitions, logic and packaged data for several popular graph/path search algorithms. Developed in Python during the course of CSPB-3202 Artificail Intelligence.

![Search_Paths](Assets/Search_Paths.png)

### Algorithm Links:
<a href="#BFS">Breadth First Search</a><br>
<a href="#DFS">Depth First Search</a><br>
<a href="#UFS">Uniform Cost Search (Djikstra's)</a><br>
<a href="#A">A\* Search</a><br>
***

### Map Data Structure
The abstracted maps below were the graphs used to develop and test the functions found in this notebook.

In [16]:
from collections import deque
from collections import OrderedDict
map_distances = dict(
    chi=OrderedDict([("det",283), ("cle",345), ("ind",182)]),
    cle=OrderedDict([("chi",345), ("det",169), ("col",144), ("pit",134), ("buf",189)]),
    ind=OrderedDict([("chi",182), ("col",176)]),
    col=OrderedDict([("ind",176), ("cle",144), ("pit",185)]),
    det=OrderedDict([("chi",283), ("cle",169), ("buf",256)]),
    buf=OrderedDict([("det",256), ("cle",189), ("pit",215), ("syr",150)]),
    pit=OrderedDict([("col",185), ("cle",134), ("buf",215), ("phi",305), ("bal",247)]),
    syr=OrderedDict([("buf",150), ("phi",253), ("new",254), ("bos",312)]),
    bal=OrderedDict([("phi",101), ("pit",247)]),
    phi=OrderedDict([("pit",305), ("bal",101), ("syr",253), ("new",97)]),
    new=OrderedDict([("syr",254), ("phi",97), ("bos",215), ("pro",181)]),
    pro=OrderedDict([("bos",50), ("new",181)]),
    bos=OrderedDict([("pro",50), ("new",215), ("syr",312), ("por",107)]),
    por=OrderedDict([("bos",107)]))

map_times = dict(
    chi=dict(det=280, cle=345, ind=200),
    cle=dict(chi=345, det=170, col=155, pit=145, buf=185),
    ind=dict(chi=200, col=175),
    col=dict(ind=175, cle=155, pit=185),
    det=dict(chi=280, cle=170, buf=270),
    buf=dict(det=270, cle=185, pit=215, syr=145),
    pit=dict(col=185, cle=145, buf=215, phi=305, bal=255),
    syr=dict(buf=145, phi=245, new=260, bos=290),
    bal=dict(phi=145, pit=255),
    phi=dict(pit=305, bal=145, syr=245, new=150),
    new=dict(syr=260, phi=150, bos=270, pro=260),
    pro=dict(bos=90, new=260),
    bos=dict(pro=90, new=270, syr=290, por=120),
    por=dict(bos=120))

![Maps Distances](Assets/Maps_Distances.png)
![Maps Times](Assets/Maps_Times.png)
![Maps Heur](Assets/Maps_Heur.png)

### Graph Data Structure
The class definition for the graph data structure used for the search algorithms.

In [81]:
## Graph class data structure...
##
class Graph:
    class Node:
        def __init__(self,value,parent=None,adjacent=[],visited=False):
            self.value = value
            self.parent = parent
            self.adjacent = adjacent
            self.visited = visited #optional: BFS...
            self.cost = float("inf") #optional: BFS/DFS...
            
        
    def __init__(self, graph=None,found=False):
        if graph == None:
            graph = {}
            
        self.graph = graph
        self.found = found
        self.state = {} #optional: BFS/DFS...
        self.expanded = [] #optional: BFS/DFS...
        
    def _list_vertices(self):
        return list(self.graph.keys())
    
    def _list_edges(self):
        edges = []
        for vertex in self.graph:
            for neighbor in self.graph[vertex].adjacent:
                if {neighbor, vertex} not in edges:
                    edges.append({vertex, neighbor})         
        return edges
    
    def addVertex(self, key):
        # check if key already exists
        if key in self.graph:
            print("Vertex already exists")
        else:
            vertex = self.Node(key)
            vertex.adjacent = list(map_distances[key]) #optional: BFS/DFS...
            self.graph[key] = vertex
            self.state[key] = vertex.cost
            
        #print("Vertex ", key, " added.")
        return vertex

    def addEdge(self, key1, key2):
        (v1, v2) = (key1, key2)
        if v1 in self.graph:
            self.graph[v1].adjacent.append(v2)
        else:
            vertex1 = self.Node(v1,[v2])
            self.addVertex(v1,vertex1)
            
        if v2 not in self.graph:
            vertex2 = self.Node(v2,[v1])
            self.addVertex(v2,vertex2)
            vertex2.parent = self.findVertex(v1)
        #print("Edge added between ", key1, " and ", key2, " added.")
        return
    
    def relax(self,u,v,weight):
        if v.cost > u.cost + weight:
            v.cost = u.cost + weight
            self.state[v.value] = v.cost
            v.parent = u
            if v.value not in self.expanded and self.found == False:
                self.expanded.append(v.value)

    def findVertex(self, key):
        if key in self.graph:
            return self.graph.get(key)
        else:
            print("Vertex not present in graph.")
            
    def findPath(self, goal):
        city = goal
        path = [city.value]
        while city.parent != None:
            path.append(city.parent.value)
            city = city.parent
        #print("Path: ",path, "\n Reversed Path: ", path.reverse())
        path.reverse()
        return path
    
    def findPathCost(self, path, step_costs):
        cost = 0
        i = 0
        while i < len(path)-1:
            v1 = path[i]
            v2 = path[i+1]
            #print("v1: ",v1, ", v2: ",v2)
            cost += step_costs[v1][v2]
            i += 1
        return cost

In [82]:
## Graph class data structure testing...
##
g = Graph()
g.addVertex("chi")
g.addVertex("det")
g.addVertex("cle")
g.addVertex("ind")

g.addEdge("chi","det")
g.addEdge("chi","cle")
g.addEdge("chi","ind")
g.addEdge("det","cle")

vertices = g._list_vertices()
edges = g._list_edges()

print("Vertices: ", vertices)
print("Edges: ", edges)

chi = g.findVertex("chi")
g.findVertex("buf")

Vertices:  ['chi', 'det', 'cle', 'ind']
Edges:  [{'det', 'chi'}, {'cle', 'chi'}, {'ind', 'chi'}, {'det', 'cle'}, {'det', 'buf'}, {'col', 'cle'}, {'pit', 'cle'}, {'cle', 'buf'}, {'col', 'ind'}]
Vertex not present in graph.


<span id="BFS"></span>
***
## Breadth First Search
Graph class definition for implementation of Breadth First Search.

![BFS Map](Assets/BFS_Path.png)

In [83]:
## Breaadth First Search Implementation...
##
def breadth_first(start, goal, state_graph, return_cost=False):
    # initialize graph with start city...
    g = Graph()
    origin = g.addVertex(start)
    #origin = g.findVertex(start)
    origin.adjacent = list(map_distances[start])
    
    # verify that start is not goal...
    if start == goal:
        return origin
    
    origin.visited = True
    
    # populate graph and establish unvisited queue...
    for city in map_distances:
        if city == start:
            continue
        else:
            g.addVertex(city)
            
    # initialize unvisited priority queue...
    q = deque([start])
    #print("Queue: ", q)
    
    while len(q) > 0:
        city = q.popleft()
        vertex = g.findVertex(city)
        print("Popped ", vertex.value, " from the queue.")
        print("Adjacent Cities: ", list(map_distances[city]),"\n")
        
        for neighbor in vertex.adjacent:
            neighbor_vertex = g.findVertex(neighbor)
            
            if neighbor_vertex.value == goal:
                neighbor_vertex.parent = vertex
                print("Path from ", origin.value, " to ", goal, " was found.")
                
                path = g.findPath(neighbor_vertex)
                
                if return_cost == False:
                    return path
                else:
                    cost = g.findPathCost(path,map_distances)
                    return (path,cost)
            
            if neighbor_vertex.visited == False:
                #print("Next City: ", neighbor_vertex.value)
                g.addEdge(vertex.value, neighbor_vertex.value)
                
                neighbor_vertex.visited = True
                neighbor_vertex.adjacent = list(map_distances[neighbor])
                neighbor_vertex.parent = vertex
                #print("Adjacent Cities: ", neighbor_vertex.adjacent)
                
                q.append(neighbor_vertex.value)
                #print("Queue: ", q)
                
    print("Path to ", goal, " was not found.")            
    return

In [84]:
print("Breadth First Search Implementation:")
(path,cost) = breadth_first("chi", "pro", map_distances,True)
print("Path: ", path, ", Cost: ", cost)

Breadth First Search Implementation:
Popped  chi  from the queue.
Adjacent Cities:  ['det', 'cle', 'ind'] 

Popped  det  from the queue.
Adjacent Cities:  ['chi', 'cle', 'buf'] 

Popped  cle  from the queue.
Adjacent Cities:  ['chi', 'det', 'col', 'pit', 'buf'] 

Popped  ind  from the queue.
Adjacent Cities:  ['chi', 'col'] 

Popped  buf  from the queue.
Adjacent Cities:  ['det', 'cle', 'pit', 'syr'] 

Popped  col  from the queue.
Adjacent Cities:  ['ind', 'cle', 'pit'] 

Popped  pit  from the queue.
Adjacent Cities:  ['col', 'cle', 'buf', 'phi', 'bal'] 

Popped  syr  from the queue.
Adjacent Cities:  ['buf', 'phi', 'new', 'bos'] 

Popped  phi  from the queue.
Adjacent Cities:  ['pit', 'bal', 'syr', 'new'] 

Popped  bal  from the queue.
Adjacent Cities:  ['phi', 'pit'] 

Popped  new  from the queue.
Adjacent Cities:  ['syr', 'phi', 'bos', 'pro'] 

Path from  chi  to  pro  was found.
Path:  ['chi', 'det', 'buf', 'syr', 'new', 'pro'] , Cost:  1124


<span id="DFS"></span>
***
## Depth First Search
Graph class definition for implementation of Depth Cost Search.

![DFS Map](Assets/DFS_Path.png)

In [85]:
## Depth First Search node visit function...
##
def dfs_visit(g, node, goal,return_cost):
    print("\nNode: ", node.value, "\nAdjacent: ", node.adjacent)
    node.visited = True

    #for neighbor in node.adjacent:
    
    queue = node.adjacent
    for i in range(len(queue)):
        if g.found == True:
            return
        
        # grab graph node for each adjacent node...
        neighbor = queue.pop()
        neighbor_vertex = g.findVertex(neighbor)
        print("Print DFS Recursive Neighbor: ", neighbor_vertex.value)
        
        # if neighbor city is the destination...
        if neighbor_vertex.value == goal:
            g.found = True
            neighbor_vertex.parent = node
            print("Path from ", g.root.value, " to ", goal, " was found.")
            return
        
        if neighbor_vertex.visited == False:
            print("Next City: ", neighbor_vertex.value)
            g.addEdge(node.value, neighbor_vertex.value)

            neighbor_vertex.visited = True
            neighbor_vertex.adjacent = list(map_distances[neighbor])
            neighbor_vertex.parent = node
            result = dfs_visit(g,neighbor_vertex,goal,return_cost)
            #if g.found == True:
                #return result

In [86]:
## Depth First Search Implementation...
##
def depth_first(start, goal, state_graph, return_cost=False):
    result = None
    found = False
    path = []
    cost = 0
    
    # initialize graph with start city...
    g = Graph()
    time = 0
    origin = g.addVertex(start)
    g.root = origin
    
    origin.adjacent = list(map_distances[start])
    origin.visited = True
    
    # verify that start is not goal, set to visited if not...
    if start == goal:
        return origin
    
    # populate graph and establish unvisited queue...
    for city in map_distances:
        if city == start:
            continue
        else:
            g.addVertex(city)
            
    # initialize unvisited priority queue...
    q = deque([start])
    
    print("Origin: ", origin.value, "\nAdjacent: ", origin.adjacent)
    
    queue = origin.adjacent
    for i in range(len(queue)):
        neighbor = queue.pop()
        # grab graph node for each adjacent node...
        neighbor_vertex = g.findVertex(neighbor)
        if neighbor_vertex.visited == False:
            neighbor_vertex.adjacent = list(map_distances[neighbor])
            neighbor_vertex.parent = origin
            dfs_visit(g,neighbor_vertex,goal,return_cost)
       
        if g.found == True:
            destination = g.findVertex(goal)
            path = g.findPath(destination)

            if return_cost == False:
                return path
            else:
                cost = g.findPathCost(path,map_distances)
                return (path,cost)

In [87]:
print("Depth First Search Implementation:")
(path,cost) = depth_first("chi", "pro", map_distances,True)
print("Path: ", path, ", Cost: ", cost)

Depth First Search Implementation:
Origin:  chi 
Adjacent:  ['det', 'cle', 'ind']

Node:  ind 
Adjacent:  ['chi', 'col']
Print DFS Recursive Neighbor:  col
Next City:  col

Node:  col 
Adjacent:  ['ind', 'cle', 'pit']
Print DFS Recursive Neighbor:  pit
Next City:  pit

Node:  pit 
Adjacent:  ['col', 'cle', 'buf', 'phi', 'bal']
Print DFS Recursive Neighbor:  bal
Next City:  bal

Node:  bal 
Adjacent:  ['phi', 'pit']
Print DFS Recursive Neighbor:  pit
Print DFS Recursive Neighbor:  phi
Next City:  phi

Node:  phi 
Adjacent:  ['pit', 'bal', 'syr', 'new']
Print DFS Recursive Neighbor:  new
Next City:  new

Node:  new 
Adjacent:  ['syr', 'phi', 'bos', 'pro']
Print DFS Recursive Neighbor:  pro
Path from  chi  to  pro  was found.
Path:  ['chi', 'ind', 'col', 'pit', 'bal', 'phi', 'new', 'pro'] , Cost:  1169


<span id="UCS"></span>
***
## Uniform Cost Search 
Graph class definition for implementation of Uniform Cost Search. The first cell contains the original UCS search algorithm from the first homework, which will be modified to A\*.

![UCS Map](Assets/UCS_Path.png)

In [102]:
## Uniform Cost Search Implementation...
##
def uniform_cost(start, goal, state_graph, return_cost=False, return_nexp=True):
    result = None
    found = False
    path = []
    cost = 0
    
    # initialize graph with start city...
    g = Graph()
    origin = g.addVertex(start)
    g.root = origin
    g.expanded.append(origin.value)
    origin.adjacent = list(map_distances[start])
    origin.visited = True
    origin.cost = 0
    origin.index = 0
    g.state[origin.value] = 0
    
    # verify that start is not goal, set to visited if not...
    if start == goal:
        return origin
    
    # populate graph and establish unvisited queue...
    pq = Frontier_PQ()
    pq.push_PQ(origin.value,0)
    
    for city in map_distances:
        if city == start:
            continue
        else:
            g.addVertex(city)
            g.state[city] = float('inf')
    
    while pq.__len__() > 0:
        nextNode = pq.pop_PQ()
        u = g.findVertex(nextNode[1])
        u.visited = True
        
        for vertex in u.adjacent:
            v = g.findVertex(vertex)
            
            if v.value == goal:
                g.found = True
            
            edge_weight = map_distances[u.value][v.value]
            g.relax(u,v,edge_weight)
            
            if v.visited == False:
                pq.replace_PQ(vertex,v.cost)
    
    if g.found == True:
        destination = g.findVertex(goal)
        path = g.findPath(destination)
        g.expanded.append(goal)

        if return_cost == False and return_nexp == False:
            return path
        elif return_cost == False and return_nexp == True:
            nodes = len(g.expanded)
            return (path,nodes)
        else:
            cost = g.findPathCost(path,map_distances)
            nodes = len(g.expanded)
            return (path,cost,nodes)   
        
    return
    # add your code here
    
    # add your code here
#path, cost = uniform_cost('chi','pit',map_distances,True)
#print(path)
#print(cost)
uniform_cost('chi', 'pro', map_distances, True, True)

(['chi', 'cle', 'buf', 'syr', 'bos', 'pro'], 1046, 13)

In [101]:
(path,cost,nodes) = uniform_cost("chi", "pro", map_distances,True,True)
print("Uniform Cost Search Implementation:\n  Path: {}\n  Distance: {}\n  Nodes Expanded: {}".format(path,cost,nodes))

Uniform Cost Search Implementation:
  Path: ['chi', 'cle', 'buf', 'syr', 'bos', 'pro']
  Distance: 1046
  Nodes Expanded: 13


<span id="A"></span>
***
## A\* Search 
Graph class definition for implementation of A\* path generation.

![A* Map](Assets/A_Path.png)

#### A\* Search Heuristic Dictionary + Function
The following helper function returns the straight-line distance for use in A\* heuristics.

In [104]:
sld_providence = dict(
    chi=833,
    cle=531,
    ind=782,
    col=618,
    det=596,
    buf=385,
    pit=458,
    syr=253,
    bal=325,
    phi=236,
    new=157,
    pro=0,
    bos=38,
    por=136)
    
def heuristic_sld_providence(state):
    return sld_providence[state]

heuristic_sld_providence("pit")

458

In [106]:
## A* Search Implementation...
##
def astar_search(start, goal, state_graph, heuristic, return_cost=False, return_nexp=False):
    '''A* search from `start` to `goal`
    start = initial state
    goal = goal state
    heuristic = function for estimated cost to goal (function name)
    return_cost = logical (True/False) for whether or not to return the total path cost
    return_nexp = logical (True/False) for whether or not to return the number of nodes expanded
    '''         
    result = None
    found = False
    path = []
    cost = 0
    
    # initialize graph with start city...
    g = Graph()
    origin = g.addVertex(start)
    g.root = origin
    #g.expanded.append(origin.value)
    origin.adjacent = list(map_distances[start])
    origin.visited = True
    origin.cost = 0
    origin.index = 0
    g.state[origin.value] = 0
    
    # verify that start is not goal, set to visited if not...
    if start == goal:
        return origin
    
    # populate graph and establish unvisited queue...
    pq = Frontier_PQ()
    pq.push_PQ(origin.value,heuristic(start))
    
    for city in map_distances:
        if city == start:
            continue
        else:
            g.addVertex(city)
            g.state[city] = float('inf')
    
    while pq.__len__() > 0:
        print("\nPQ is non-zero...")
        pq.__str__()
        nextNode = pq.pop_PQ()
        print("Expanding ", nextNode[1])
        u = g.findVertex(nextNode[1])
        u.visited = True
        g.expanded.append(u.value)
        
        if u.value == goal:
            print("\n", u.value, "found!!\n")
            break
        
        for vertex in u.adjacent:
            v = g.findVertex(vertex)
            
            if v.value == goal:
                #print("\n", v.value, "found!!\n")
                g.found = True
            
            edge_weight = map_distances[u.value][v.value]
            h_weight = heuristic(v.value)
            comb_weight = edge_weight + h_weight
            print("Relaxing node: ", v.value)
            
            g.relax(u,v,comb_weight)
            
            if v.visited == False:
                #straight_dist = heuristic(v.value)
                #print("Distance to ", v.value, "is ", straight_dist)
                pq.replace_PQ(vertex,comb_weight)
    
    if g.found == True:
        destination = g.findVertex(goal)
        path = g.findPath(destination)
        #g.expanded.append(goal)

        if return_cost == False and return_nexp == False:
            return path
        elif return_cost == False and return_nexp == True:
            nodes = len(g.expanded)
            print("Nodes: ", g.expanded)
            return (path,nodes)
        elif return_cost == True and return_nexp == False:
            cost = g.findPathCost(path,map_distances)
            return (path,cost)
        else:
            cost = g.findPathCost(path,map_distances)
            nodes = len(g.expanded)
            print("Nodes: ", g.expanded)
            return (path,cost,nodes)   
    return

astar_search('chi', 'pro', map_distances, heuristic_sld_providence, True, True)


PQ is non-zero...
[[833, 'chi']]
Expanding  chi
Relaxing node:  det
Relaxing node:  cle
Relaxing node:  ind

PQ is non-zero...
[[876, 'cle'], [879, 'det'], [964, 'ind']]
Expanding  cle
Relaxing node:  chi
Relaxing node:  det
Relaxing node:  col
Relaxing node:  pit
Relaxing node:  buf

PQ is non-zero...
[[574, 'buf'], [592, 'pit'], [765, 'det'], [964, 'ind'], [762, 'col']]
Expanding  buf
Relaxing node:  det
Relaxing node:  cle
Relaxing node:  pit
Relaxing node:  syr

PQ is non-zero...
[[403, 'syr'], [673, 'pit'], [852, 'det'], [964, 'ind'], [762, 'col']]
Expanding  syr
Relaxing node:  buf
Relaxing node:  phi
Relaxing node:  new
Relaxing node:  bos

PQ is non-zero...
[[350, 'bos'], [673, 'pit'], [411, 'new'], [964, 'ind'], [762, 'col'], [852, 'det'], [489, 'phi']]
Expanding  bos
Relaxing node:  pro
Relaxing node:  new
Relaxing node:  syr
Relaxing node:  por

PQ is non-zero...
[[50, 'pro'], [243, 'por'], [372, 'new'], [673, 'pit'], [762, 'col'], [852, 'det'], [489, 'phi'], [964, 'ind']]


(['chi', 'cle', 'buf', 'syr', 'bos', 'pro'], 1046, 16)